In [1]:
import torch
import gc
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

d:\Uni\Semester 8\Gen AI\Project\Lyrical\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('../data/processed/lyrics_processed.csv')

In [3]:
train_data = []
for i in range(len(data)):
    data_elem = data.iloc[i]
    song = data_elem['song']
    artist = data_elem['artist']
    year = data_elem['year']
    genre = data_elem['genre']
    lyrics = data_elem['lyrics']
    
    train_data.append({"input_text": f"Give me the lyrics of a song that was made by {artist} and was releasd in {year} and is of the {genre} genre.", "label_text": lyrics})

In [ ]:
train_data, test_data = train_data[:int(0.9*len(train_data))], train_data[int(0.9*len(train_data)):]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

d:\Uni\Semester 8\Gen AI\Project\Lyrical\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Uni\Semester 8\Gen AI\Project\Lyrical\venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Syed\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to 

In [10]:
tokenized_data = tokenizer([d["input_text"] for d in train_data], padding=True, truncation=True, return_tensors='pt')
input_ids = tokenized_data.input_ids
attention_mask = tokenized_data.attention_mask

label_ids = tokenizer([d["label_text"] for d in train_data], padding=True, truncation=True, return_tensors='pt').input_ids

In [5]:
batch_size = 8

dataset = TensorDataset(input_ids, attention_mask, label_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [6]:
def train(model, dataloader, optimizer, device, epoch):
    torch.cuda.empty_cache()
    gc.collect()
    model.train()
    total_loss = 0.0
    for idx, batch in enumerate(dataloader):
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(f"Epoch {epoch+1} - Batch {idx*batch_size}/{len(dataloader)} - Loss: {loss.item()}")
        
        model.save_pretrained('../models/flan_t5_large')
        
        del input_ids
        del attention_mask
        del labels
        torch.cuda.empty_cache()
        gc.collect()
    del dataloader
    torch.cuda.empty_cache()
    gc.collect()
        
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} Complete! - Average Loss: {avg_loss}")

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

learning_rate = 1e-4
lr_scheduler_step = 1
lr_scheduler_gamma = 0.9
num_epochs = 10

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_scheduler_step, gamma=lr_scheduler_gamma)

In [13]:
torch.cuda.empty_cache()
gc.collect()
for epoch in range(num_epochs):
    train(model, dataloader, optimizer, device, epoch)
    scheduler.step()
    model.save_pretrained('../models/flan_t5_large')
    torch.cuda.empty_cache()
    gc.collect()
model.save_pretrained('../models/flan_t5_large')
torch.cuda.empty_cache()
gc.collect()

Epoch 1 - Batch 0/5351 - Loss: 10.681533813476562
Epoch 1 - Batch 8/5351 - Loss: 14.086360931396484
Epoch 1 - Batch 16/5351 - Loss: 12.969615936279297
Epoch 1 - Batch 24/5351 - Loss: 13.816482543945312
Epoch 1 - Batch 32/5351 - Loss: 10.84506893157959
Epoch 1 - Batch 40/5351 - Loss: 13.249403953552246
Epoch 1 - Batch 48/5351 - Loss: 11.773417472839355
Epoch 1 - Batch 56/5351 - Loss: 13.160143852233887
Epoch 1 - Batch 64/5351 - Loss: 8.932929992675781
Epoch 1 - Batch 72/5351 - Loss: 8.956905364990234
Epoch 1 - Batch 80/5351 - Loss: 10.419461250305176
